In [ ]:
import pandas as pd
import math
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn import svm
from sklearn.linear_model import SGDRegressor
from sklearn import tree
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import pickle

import os,re,sys
from flask import Flask, abort, jsonify, request, session


In [ ]:
def celsius(df,columns):
    for column in columns:
        t=column.split('_')
        del t[len(t)-1]
        t.append('c')
        newcol='_'.join(t)
        df[newcol]=df[column].apply(lambda x : x-273.15 )
    return df

In [ ]:
def convertlist():
    columns=['reanalysis_dew_point_temp_k','reanalysis_air_temp_k','reanalysis_avg_temp_k','reanalysis_min_air_temp_k','reanalysis_max_air_temp_k','reanalysis_min_air_temp_k']
    return columns

In [ ]:
def preprocess(df):
    df['reanalysis_tdtr_c']=df['reanalysis_max_air_temp_c']-df['reanalysis_min_air_temp_c']
    return df


In [ ]:
def drop(df):
    df=df.drop(['week_start_date','reanalysis_dew_point_temp_k','reanalysis_air_temp_k','reanalysis_avg_temp_k','reanalysis_min_air_temp_k','reanalysis_max_air_temp_k','reanalysis_min_air_temp_k','reanalysis_tdtr_k'],axis=1)
    return df

In [ ]:
def imputevalues(df):
   # print(len(df))
    wantedcolumnlist=[]
    columnlist=df.columns.tolist()
    #print(len(columnlist))
    for i in range(len(columnlist)):
        columnname=columnlist[i]
        nan_rows = df[columnname].isnull().sum()
     #   print(i,nan_rows)
        nullcount=(nan_rows/len(df))*100
        #print(nullcount)
        if nullcount!=0:
            df[columnname]=df[columnname].fillna(df[columnname].mean()) 
        else:
            pass    
      
    
    return df

In [ ]:
def is_chances(features):
   
    coolest = []
    warmest=[]
    normal=[]
    
    for observation in features['station_avg_temp_c']:
       # print(observation)
        if int(observation) > 28:
            warmest.append(1)
        else:
            warmest.append(0)
            
    for observation in features['station_avg_temp_c']:
        if observation < 24:
            coolest.append(1)
        else:
            coolest.append(0)  
            
    for observation in features['station_avg_temp_c']:
        if int(observation) > 24 & int(observation) <=28:
            normal.append(1)
        else:
            normal.append(0)

    return warmest,coolest,normal

In [ ]:
disease_mode = pickle.load( open( "disease_mode.pickle", "rb" ) )
disease_label_encoder = pickle.load( open( "disease_label_encoder.pickle", "rb" ) )

In [ ]:
app = Flask(__name__)

In [ ]:
@app.route('/audio', methods = ['POST'])
def audio_identification():
    
    try:
        
        data = request.get_json(force = True)
        
        filepath = data['filepath']
        #print(filepath)
        test=pd.read_csv(filepath)
        #print(test.head(5))
        
        columns=convertlist()
        test=celsius(test,columns)
        test=preprocess(test)
        test=drop(test)
        test=imputevalues(test)
        warmest,coolest,normal = is_chances(test)
        
        test['verywarm']=warmest
        test['verycool']=coolest
        test['normal']=normal
       
        test['city'] = disease_label_encoder.transform(test['city'])
          
        pred = disease_mode.predict(test).astype(int)
        
        pred = pred.tolist()
        
    except Exception as e:
        return jsonify ({'Error':str(e),'status':'False'})
        
    else:
        return jsonify({'prediction':pred})

In [ ]:
app.run("0.0.0.0", port = 7001, debug = True, use_reloader = False,threaded=True)